In [ ]:
import pandas as pd
import numpy as np
import math
from qiskit import QuantumCircuit
from qiskit import Aer, execute
import os

In [ ]:
data = pd.read_csv('Montabilidad_to_Quantum.csv',delimiter=';', encoding='latin-1')

columnas_a_quitar = ['file_type','metadata_path', 'metadata_date', 'metadata_size', 'query_key', 'text', 'Expediente', 'Tipo_vehículo', 'Código', 'N_Produccion', 'Fecha_Tecnica', 'metadata_file']
data = data.drop(columnas_a_quitar, axis=1)

In [ ]:
def preparacion_vectores_train_test(data):
    vectores_train = data[data['Set'] == 'Train']
    vectores_test = data[data['Set'] == 'Test']
    #vectores_test = data[data['N_Pedido'] == '5-30033330']
    #vectores_train = data[data['N_Pedido'] == '5-30033330']
    #vectores_train.iloc[0:,4:-3] = 0
    #vectores_train.iloc[0:,3:9] = 1
    return (vectores_test, vectores_train)

In [ ]:
def normalizar_vector(vector):
    norma = np.linalg.norm(vector)
    return vector/norma

In [ ]:
def padding(vec, qubits):
    num_componentes_relleno = 2**qubits - len(vec)
    return np.pad(vec, (0, num_componentes_relleno), mode='constant', constant_values = 0)
#vec = np.array([1, 2, 3])
#padding(vec, 2)

In [ ]:
def inner_prod(vec1, vec2):
    #first check lengths are equal
    if len(vec1) != len(vec2):
        raise ValueError('Lengths of states are not equal')

    numero_componentes = len(vec1)
    nqubits = math.ceil(np.log2(numero_componentes))

    vec1 = padding(vec1, nqubits)
    vec2 = padding(vec2, nqubits)

    circ = QuantumCircuit(nqubits+1,1)
    vec = np.concatenate([vec1,vec2])/np.sqrt(2)

    circ.initialize(vec, range(nqubits+1))
    circ.h(nqubits)
    circ.measure(nqubits,0)

    backend = Aer.get_backend('qasm_simulator')
    job = execute(circ, backend, shots=20000)

    result = job.result()
    outputstate = result.get_counts(circ)

    if ('0' in outputstate.keys()):
        m_sum = float(outputstate["0"])/20000
    else:
        m_sum = 0

    inner_product = 2*m_sum-1
    return inner_product

In [ ]:
def correlacion_vectores(vector_1, vector_2):
    test_normalizado = normalizar_vector(vector_1.iloc[ :-3].to_numpy())
    train_normalizado = normalizar_vector(vector_2.iloc[ :-3].to_numpy())
    #print(test_normalizado, train_normalizado)
    return inner_prod(test_normalizado, train_normalizado)

In [ ]:
#apply(lambda x: x > 0).astype(int)
def itera_por_matriz_cuantica(vectores_test, vectores_train):
  for index_test, test in vectores_test.iterrows():
    for index_train, train in vectores_train.iterrows():
      correlacion_entre_variables = correlacion_vectores(test, train)
      yield [f'test_{index_test}', f'train_{index_train}', correlacion_entre_variables, test['Categoria'], test['N_Pedido']]

In [47]:
def vector_test_similar(exotico):
    vectores_test = data[data['Set'] == 'Test']
    for index_test, test in vectores_test.iterrows():
        if not test.equals(exotico):
            correlacion_entre_variables = correlacion_vectores(test, exotico)
            if correlacion_entre_variables != 1:
                yield [correlacion_entre_variables,test['N_Pedido']]

In [54]:
def diferencias_componentes(exotico, similar):
    binarizados_serie1 = exotico.apply(lambda x: 1 if x != 0 else 0)
    binarizados_serie2 = similar.apply(lambda x: 1 if x != 0 else 0)

    # Identificar las columnas que tienen valores diferentes
    diferencias = binarizados_serie1.index.symmetric_difference(binarizados_serie2.index)
    
    return list(diferencias)

In [56]:
resultados_por_categoria = {}
columnas = ['Nombre_Test', 'Nombre_Train', 'Correlacion', 'Categoria', 'N_Pedido']
exoticos = pd.DataFrame()

categorias = data.groupby('Categoria')

for nom_categoria, categoria in categorias:
    if nom_categoria == 'BASE':
         vectores_test, vectores_train = preparacion_vectores_train_test(categoria) 
         
         resultados_correlacion = pd.DataFrame(itera_por_matriz_cuantica(vectores_test, vectores_train), columns=columnas)
         resultados_correlacion_ordenado = resultados_correlacion.sort_values(by='Correlacion')

         exotico = pd.Series(vectores_test[vectores_test['N_Pedido'] == resultados_correlacion_ordenado.iloc[0]['N_Pedido']].iloc[0])
         
         resultados_similares = pd.DataFrame(vector_test_similar(exotico), columns=['Correlacion', 'N_Pedido']).sort_values(by='Correlacion')
         test_similar = pd.Series(data[data['N_Pedido'] == resultados_similares.iloc[-1]['N_Pedido']].iloc[0])

         diferencias = diferencias_componentes(exotico, test_similar)

         #exoticos = pd.concat([exoticos, pd.DataFrame(data[data['N_Pedido'] == exotico.iloc[0]['N_Pedido']])], ignore_index=True)
    
         #nombre_resultado = f'resultados_{nom_categoria}'
         #resultados_por_categoria[nombre_resultado] = resultados_correlacion

         #resultados_csv(ruta_guardado,resultados_correlacion_ordenado,nombre_resultado)
         
         #display(resultados_correlacion_ordenado)

KeyboardInterrupt: 